In [4]:
import cv2
from deepface import DeepFace
import datetime
import time
from collections import defaultdict

# Open the webcam
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

# Track unique emotions and their frequencies
logged_emotions = set()
emotion_counts = defaultdict(int)
last_logged_time = time.time()

# Timer for frame skipping to improve performance
frame_skip = 3  # Process every 3rd frame

print("Emotion detection started. Press 'q' to quit.")

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Skip frames to improve performance
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    # Resize frame for faster processing
    frame_resized = cv2.resize(frame, (640, 480))

    try:
        # Analyze emotions with 'mtcnn' detector
        analysis = DeepFace.analyze(frame_resized, actions=['emotion'], detector_backend='mtcnn', enforce_detection=False)

        # Get dominant emotion
        if isinstance(analysis, list):
            dominant_emotion = analysis[0].get('dominant_emotion', None)
        else:
            dominant_emotion = analysis.get('dominant_emotion', None)

        if dominant_emotion:
            print("Detected Emotion:", dominant_emotion)
            emotion_counts[dominant_emotion] += 1

            # Log every detected emotion
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_line = f"{timestamp} - Frame Emotion: {dominant_emotion}\n"
            with open("emotion_report.txt", "a") as log_file:
                log_file.write(log_line)

            # Log unique emotions every 10 seconds
            current_time = time.time()
            if current_time - last_logged_time >= 10 and dominant_emotion not in logged_emotions:
                log_entry = f"{timestamp} - Unique Detected Emotion: {dominant_emotion}\n"
                with open("emotion_report.txt", "a") as log_file:
                    log_file.write(log_entry)

                print(f"Logged unique emotion: {dominant_emotion}")
                logged_emotions.add(dominant_emotion)
                last_logged_time = current_time

    except Exception as e:
        print(f"Error analyzing emotions: {e}")
        dominant_emotion = None

    # Display the frame with emotion
    cv2.putText(frame_resized, f"Emotion: {dominant_emotion if dominant_emotion else 'Unknown'}", 
                (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Emotion Detection", frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# End session and log the most common emotion
most_frequent_emotion = max(emotion_counts, key=emotion_counts.get, default=None)
with open("emotion_report.txt", "a") as log_file:
    log_file.write("\nSession ended at: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
    if most_frequent_emotion:
        log_file.write(f"Mostly {most_frequent_emotion}\n")

cap.release()
cv2.destroyAllWindows()

print("Emotion detection session ended. Check 'emotion_report.txt' for the results.")


Emotion detection started. Press 'q' to quit.
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: sad
Detected Emotion: angry
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: sad
Detected Emotion: sad
Detected Emotion: sad
Detected Emotion: sad
Detected Emotion: neutral
Detected Emotion: surprise
Detected Emotion: angry
Detected Emotion: angry
Detected Emotion: fear
Detected Emotion: sad
Detected Emotion: sad
Detected Emotion: angry
Detected Emotion: fear
Detected Emotion: sad
Detected Emotion: fear
Detected Emotion: happy
Detected Emotion: neutral
Detected Emotion: neutral
Detected Emotion: sad
Detected Emotion: neutral
Detected Emotion: neutral
Detected Emotion: fear
Detected Emotion: sad
Detected Emotion: neutral
Detected Emotion: